In [ ]:
!wget -nc https://osf.io/8qcdp/download -O ObjectOrientationData.mat
!wget -nc https://osf.io/49qeb/download -O InanimateObjectsData.mat
!wget -nc https://osf.io/x9dz4/download -O InanimateObjects.zip
!unzip InanimateObjects.zip
!mkdir -p Stimuli 
!mv InanimateObjects ./Stimuli/InanimateObjects
!wget -c https://raw.githubusercontent.com/harvard-visionlab/sroh/main/2022/feature_extractor.py

!wget -nc https://www.dropbox.com/s/hq3n4ti12t9mm7g/ExampleRDMS_EarlyV_conv_block_2.1.pth.tar?dl=0

# Getting Brain Data


In [ ]:
import os
import scipy.io as sio
import numpy as np
import torch


def load_brain_data(dataset, brain_regions=['EarlyV', 'pOTC', 'aOTC']):
    assert dataset in ['InanimateObjects', 'ObjectOrientation']
    path = f'{dataset}Data.mat'
    D = sio.loadmat(path, struct_as_record=False, squeeze_me=True)
    rdms = {r: D['rdms'].__dict__[r] for r in brain_regions}
    betas = {r: D['betas'].__dict__[r] for r in brain_regions}
    reliability = {r: D['reliability'].__dict__[r] for r in brain_regions}
    image_names = [f.strip() for f in D['image_names']]
    return rdms, betas, reliability, image_names

In [ ]:
rdms, betas, reliability, image_names = load_brain_data('InanimateObjects')

# RidgeCV

In [ ]:
import numpy as np
from fastprogress.fastprogress import progress_bar
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import r2_score
from pdb import set_trace
from copy import deepcopy
from tqdm import tqdm
import sklearn

default_alphas = np.concatenate([np.array([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]), np.logspace(1, 5, 50)])

def leave_one_out_ridge(X, y, alphas=default_alphas, fit_intercept=True, normalize=True, mb=None):
    '''
        Construct predicted brain patterns by training on N-1 items, 
        and then predicting the held out item.
        
        X: model responses [numItems x numFeatures]
        y: brain responses [numItems x numVoxels]
    '''
    n_items, n_features = X.shape
    n_voxels = y.shape[1]
    y_pred = np.zeros(y.shape)

    ALPHAS = []
    COEF_M = np.zeros((n_voxels, n_features))
    INTERCEPT = []
    for iter_count, test_idx in enumerate(progress_bar(range(n_items), total=n_items)):
        train_idxs = np.ones(n_items) == True
        train_idxs[test_idx] = False
        test_idxs = ~train_idxs  

        clf = RidgeCV(alphas=alphas, fit_intercept=fit_intercept)

        if normalize:
            scaler = StandardScaler()
            scaler.fit(X[train_idxs])
            X_train = scaler.transform(X[train_idxs])
            X_test = scaler.transform(X[test_idxs])        
        else:
            X_train = X[train_idxs]
            X_test = X[test_idxs]
              
        clf.fit(X_train, y[train_idxs])
        y_pred[test_idxs] = clf.predict(X_test)
        
        ALPHAS.append(clf.alpha_)
        COEF_M += clf.coef_
        INTERCEPT.append(clf.intercept_)

    ALPHAS = np.stack(ALPHAS)
    COEF_M /= iter_count
    INTERCEPT = np.stack(INTERCEPT)
    R2 = r2_score(y, y_pred, multioutput='raw_values')
    
    return {
        "n_items": n_items,
        "n_features": n_features,
        "n_voxels": y.shape[1],
        "ALPHAS": ALPHAS,
        "COEF_M": COEF_M,
        "INTERCEPT": INTERCEPT,
        "R2": R2,
        "y_pred": y_pred
    }

In [ ]:
import torch
from torchvision import models, transforms 
from PIL import Image 
from natsort import natsorted 
from glob import glob 
from pathlib import Path 
from feature_extractor import FeatureExtractor

def prepare_images(dataset='InanimateObjects', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):

    # standard imagenet normalization
    normalize = transforms.Normalize(mean=mean, std=std)

    transform = transforms.Compose([
        lambda x: Image.open(x),    # use PIL to open the image
        transforms.Resize(224),     # resize shorted edge to 224 pixels
        transforms.CenterCrop(224), # center crop if not square
        transforms.ToTensor(),      # convert from RGB (HxWxC) to channels first torch tensor [CxHxW]
        normalize                   # normalize by imagenet stats
    ])
    files = natsorted(glob(f'./Stimuli/{dataset}/*.jpg'))
    file_names = [Path(f).name for f in files] 
    imgs = torch.stack([transform(f) for f in files])

    return imgs

def fit_encoding_model(betas, layer_name, model_name='alexnet',
                       dataset='InanimateObjects', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
          
    print("==> prepare images")
    imgs = prepare_images(dataset=dataset, mean=mean, std=std)
    
    print("==> load pretrained model")
    model = models.__dict__[model_name](pretrained=True)

    print("==> extract activation map for the given layer")
    pred_rdms = {}
    feat_rdms = {}
    model.eval()   # <-- very important, freeze normalization stats, no dropout etc.
    with FeatureExtractor(model, [layer_name]) as extractor:
        features = extractor(imgs)
        for layer_name,feat in features.items():
            # retain spatial information, but flatten rows into a 1D feature vector
            X = torch.flatten(feat, 1)
            feat_rdm = 1 - np.corrcoef(X)
            feat_rdms[layer_name] = feat_rdm
            
            print(f"==> fitting ridge regression model ({layer_name}) (numFeatures={X.shape[1]})")
            results = leave_one_out_ridge(X, betas, fit_intercept=True, normalize=False)
            
            # compute the predicted neural RDM
            pred_rdm = 1 - np.corrcoef(results['y_pred'])
            pred_rdms[layer_name] = pred_rdm
              
            # now do something with the rdms, e.g., save them for our split-half analysis
          
    return pred_rdms, feat_rdms, results

In [ ]:
rdms.keys()

In [ ]:
rdms['EarlyV'].shape

In [ ]:
sub_betas = betas['EarlyV'][0].transpose()
sub_betas.shape

In [ ]:
model = models.alexnet(pretrained=True)

In [ ]:
pred_rdms, feat_rdms, results = fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.1',
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])

In [ ]:
plt.hist(np.abs(np.mean(coef_m, axis=0)))

In [ ]:
import matplotlib.pyplot as plt
coef_m = results['COEF_M']
#abs_coef_m = np.abs(coef_m)
x = np.abs(np.mean(abs_coef_m, axis=0))
plt.xticks([])
plt.yticks([])
plt.xlabel('weight values', size=15)
plt.ylabel('frequency of features', size=15)
plt.hist(np.abs(np.mean(coef_m, axis=0)), color='darkorange', alpha=.5)
plt.savefig('ex_weights.png', dpi=300)

In [ ]:
#should chagne this - make keys the layers - and should aleady be subx72x72 array
print("pred_rdms.keys():", pred_rdms.keys())

In [ ]:
#from load_brain_data - single sub
print("rdms.keys:", rdms.keys())
rdms['EarlyV'][1].shape

In [ ]:
#from fit_encoding_model - single sub
print("pred_rdm.keys:", pred_rdms.keys())
pred_rdms['classifier.5'].shape

In [ ]:
#from fit_encoding_model - single sub
print("feat_rdm.keys:", feat_rdms.keys())
feat_rdms['classifier.5'].shape

#Step 1: Outputs like Paper?

In [ ]:
#group-half RSA
import torch
import numpy as np
import pandas as pd
from itertools import combinations
from copy import deepcopy
from scipy.stats import pearsonr
from fastprogress.fastprogress import progress_bar

def get_split_halves(N):
    subjects = list(range(0,N))
    splits = []
    count = 0
    for subsetA in combinations(subjects, N//2):
        subsetA = list(subsetA)
        subsetB = list(np.setdiff1d(subjects, subsetA))
        assert len(np.setdiff1d(subsetA,subsetB)) == len(subsetA), "oops"
        assert len(np.setdiff1d(subsetB,subsetA)) == len(subsetB), "oops"
        assert (len(subsetA) + len(subsetB)) == N, f"oops, total should be {N}"
        splits.append((subsetA,subsetB))
        count+=1
    
    return splits[0:len(splits)//2] if N%2==0 else splits

def compute_avg_rdm(rdms):
        
    # convert dissimilarity matrix back to similarity matix (-1.0 to 1.0)
    RSMS = 1 - rdms

    # fisherz transform for averaging (gives a group similarity matrix, values still z-transformed)
    group_zrsm = fisherz(RSMS).mean(axis=0, keepdims=True)

    # compute group rdm (1 - fisherz_inv(ZRSM))
    avg_rdm = 1 - fisherz_inv(group_zrsm)
    
    return avg_rdm

def get_rdm_subset(rdms, brain_region, idxs):
    rdms = deepcopy(rdms)
    return rdms[brain_region][idxs]

def fisherz(r, eps=1e-5):
    return np.arctanh(r-eps)

def fisherz_inv(z):
    return np.tanh(z)

def compute_adjusted_ci(scores, CI=1.96):
    scores = np.array(scores)
    N = len(scores)
    mean = scores.mean()
    var = scores.var(ddof=1)    

    # sem = sd / np.sqrt(N)
    # to adjust for non-indepndence of split halves, divide by np.sqrt( (N + n2/n1)/N )
    # where n1 = number of train samples, n2 = number of test samples, in our case
    # n1 = 1 (the single group TrainRDM), and n2 = 1 (the single TestRDM)
    sem = np.sqrt( (1/N+1/1) * var)
    ci = sem * CI
    lower = mean - ci
    upper = mean + ci

    return mean, lower, upper

def compute_fisherz_ci(scores):
    scores = np.array(scores)
    out = compute_adjusted_ci(fisherz(scores))
    return [fisherz_inv(o) for o in out]

def compare_rdms(target_rdm, model_rdm):
    assert target_rdm.shape == model_rdm.shape, "oops, rdms must have same shape!"
    
    target = np.array(target_rdm[np.triu_indices(target_rdm.shape[0], k=1)])
    predicted = np.array(model_rdm[np.triu_indices(model_rdm.shape[0], k=1)])

    return pearsonr(target, predicted)[0]

def update_df(df, brain_region, model_name, layer_name, split_num, split_idxs, group, sim):
    
    df = df.append({
        "analysis": "group_halves_rsa",
        "model_name": model_name,
        "dataset": "InanimateObjects",
        "brain_region": brain_region,
        "layer_name": layer_name,
        "split_num": split_num,
        "split_idxs": str(split_idxs),
        "group": group,        
        "pearsonr": sim,
    }, ignore_index=True)

    return df

Sample Data

In [ ]:
#sample data
data = torch.load('ExampleRDMS_EarlyV_conv_block_2.1.pth.tar?dl=0')
print(data.keys())
brain_region = data['brain_region']
layer_name = data['layer_name']
print("brain region:", brain_region)
print("layer name:", layer_name)
sample_neural_rdms = data['true_rdms']
sample_pred_rdms = data['pred_rdms']
sample_feat_rdms = data['feat_rdms']
sample_neural_rdms.shape
sample_pred_rdms.shape
sample_feat_rdms.shape
#10,72,72

In [ ]:
#sample data
N = sample_pred_rdms.shape[0]
splits = get_split_halves(N)
print(f"N={N}, num_splits={len(splits)}")

In [ ]:
#sample_data - pred to neural
model_name = "ipcl_alexnet_gn"
df = pd.DataFrame(columns=['analysis','model_name','dataset','brain_region',
                           'layer_name', 'split_num', 'split_idxs', 'group', 'pearsonr'])

for split_num, (group1, group2) in enumerate(progress_bar(splits)):
    idx_group1 = np.array(group1)
    idx_group2 = np.array(group2)                                

    brain_rdm1 = compute_avg_rdm(sample_neural_rdms[idx_group1]).squeeze()
    brain_rdm2 = compute_avg_rdm(sample_neural_rdms[idx_group2]).squeeze()
    
    pred_rdm1 = compute_avg_rdm(sample_pred_rdms[idx_group1]).squeeze()
    pred_rdm2 = compute_avg_rdm(sample_pred_rdms[idx_group2]).squeeze()                                

    sim1 = compare_rdms(brain_rdm1, pred_rdm1)
    sim2 = compare_rdms(brain_rdm2, pred_rdm2)

    # record the results
    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group1, 'group1', sim1)

    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group2, 'group2', sim2)
df

In [ ]:
#sample_data - pred to neural
mean, lower, upper = compute_fisherz_ci(df.pearsonr)
print(f"mean={mean:3.3f}, 95% CI=[{lower:3.3f},{upper:3.3f}]")

In [ ]:
#sample_data - feat to neural
model_name = "ipcl_alexnet_gn"
df = pd.DataFrame(columns=['analysis','model_name','dataset','brain_region',
                           'layer_name', 'split_num', 'split_idxs', 'group', 'pearsonr'])

for split_num, (group1, group2) in enumerate(progress_bar(splits)):
    idx_group1 = np.array(group1)
    idx_group2 = np.array(group2)                                

    brain_rdm1 = compute_avg_rdm(sample_neural_rdms[idx_group1]).squeeze()
    brain_rdm2 = compute_avg_rdm(sample_neural_rdms[idx_group2]).squeeze()
    
    feat_rdm1 = compute_avg_rdm(sample_feat_rdms[idx_group1]).squeeze()
    feat_rdm2 = compute_avg_rdm(sample_feat_rdms[idx_group2]).squeeze()                                

    sim1 = compare_rdms(brain_rdm1, feat_rdm1)
    sim2 = compare_rdms(brain_rdm2, feat_rdm2)

    # record the results
    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group1, 'group1', sim1)

    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group2, 'group2', sim2)
df

In [ ]:
#sample_data - feat to neural
mean, lower, upper = compute_fisherz_ci(df.pearsonr)
print(f"mean={mean:3.3f}, 95% CI=[{lower:3.3f},{upper:3.3f}]")

My Data

In [ ]:
#can do fit encoding model in this for loop - and dont have to save dicts
subs = [0,1,2,3,4,5,6,7,8,9]
layer_name='classifier.5'
for sub in subs:
  print("\n\nSUB:", sub)
  sub_betas = betas['EarlyV'][sub].transpose()
  pred_rdms, feat_rdms, results = fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
  
  pred_rdm = pred_rdms[layer_name].reshape((1,72,72))
  feat_rdm = feat_rdms[layer_name].reshape((1,72,72))
  if sub == 0:
    pred_rdms1 = pred_rdm   #pred_rdms1 so that pred_rdms doesnt get relabeled!
    feat_rdms1 = feat_rdm   #feat_rdm1 so that feat_rdms doesnt get relabeled!
  else:
    pred_rdms1 = np.concatenate((pred_rdms1, pred_rdm), axis=0)
    feat_rdms1 = np.concatenate((feat_rdms1, feat_rdm), axis=0)



In [ ]:
pred_rdms1.shape
feat_rdms1.shape
rdms['EarlyV'].shape
#10,72,72 - 10 subs, each has 72x72 rdm (num items)

In [ ]:
#neural and model rdms - no lesions
brain_region = 'EarlyV'
layer_name = "classifier.5"
neural_rdms = rdms[brain_region]
pred_rdms1 #pred_rdms1 so that pred_rdms doesnt get relabeled!
feat_rdms1 #feat_rdm1 so that feat_rdms doesnt get relabeled!
assert pred_rdms1.shape == neural_rdms.shape, "oops, expected same shapes"

In [ ]:
# iterate over all possible splits (for 10 subjects, 126 unique splits of the data, 252 unique groups total)
N = pred_rdms1.shape[0]
splits = get_split_halves(N)
print(f"N={N}, num_splits={len(splits)}")

In [ ]:
#pred1 to neural
df = pd.DataFrame(columns=['analysis','model_name','dataset','brain_region',
                           'layer_name', 'split_num', 'split_idxs', 'group', 'pearsonr'])

model_name = 'alexnet'
for split_num, (group1, group2) in enumerate(progress_bar(splits)):
    idx_group1 = np.array(group1)
    idx_group2 = np.array(group2)                                

    brain_rdm1 = compute_avg_rdm(neural_rdms[idx_group1]).squeeze()
    brain_rdm2 = compute_avg_rdm(neural_rdms[idx_group2]).squeeze()
    
    pred_rdm1 = compute_avg_rdm(pred_rdms1[idx_group1]).squeeze()
    pred_rdm2 = compute_avg_rdm(pred_rdms1[idx_group2]).squeeze()                                

    sim1 = compare_rdms(brain_rdm1, pred_rdm1)
    sim2 = compare_rdms(brain_rdm2, pred_rdm2)

    # record the results
    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group1, 'group1', sim1)

    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group2, 'group2', sim2)
df

In [ ]:
#pred1 to neural
mean, lower, upper = compute_fisherz_ci(df.pearsonr)
print(f"mean={mean:3.3f}, 95% CI=[{lower:3.3f},{upper:3.3f}]")

In [ ]:
#feat1 to neural
df = pd.DataFrame(columns=['analysis','model_name','dataset','brain_region',
                           'layer_name', 'split_num', 'split_idxs', 'group', 'pearsonr'])

model_name = 'alexnet'
for split_num, (group1, group2) in enumerate(progress_bar(splits)):
    idx_group1 = np.array(group1)
    idx_group2 = np.array(group2)                                

    brain_rdm1 = compute_avg_rdm(neural_rdms[idx_group1]).squeeze()
    brain_rdm2 = compute_avg_rdm(neural_rdms[idx_group2]).squeeze()
    
    feat_rdm1 = compute_avg_rdm(feat_rdms1[idx_group1]).squeeze()
    feat_rdm2 = compute_avg_rdm(feat_rdms1[idx_group2]).squeeze()                                

    sim1 = compare_rdms(brain_rdm1, feat_rdm1)
    sim2 = compare_rdms(brain_rdm2, feat_rdm2)

    # record the results
    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group1, 'group1', sim1)

    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group2, 'group2', sim2)
df

In [ ]:
#feat1 to neural
mean, lower, upper = compute_fisherz_ci(df.pearsonr)
print(f"mean={mean:3.3f}, 95% CI=[{lower:3.3f},{upper:3.3f}]")

In [ ]:
#save and dowloading df as csv
df.to_csv(brain_region+"_"+model_name+"_"+layer_name+"_df")
#from google.colab import files
#files.download(brain_region+"_"+model_name+"_"+layer_name)

# Step 2: Cherry Pick a well Predicted Voxel

In [ ]:
#dims of coef_m
sub_betas = betas['EarlyV'][1].transpose()
pred_rdms, feat_rdms, results = fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
results['n_voxels'], results['n_features']
results['COEF_M'].shape

In [ ]:
#get n number of well predicted voxels - list format
def get_n_well_predicted_voxs(r2, num_vox):
  vox_idxs = []
  for i in range(1, num_vox+1):  
    r2_copy = r2.copy()
    r2_sorted = np.sort(r2_copy)
    max_vox_loc = np.argwhere(r2 == r2_sorted[-i])
    max_vox = max_vox_loc[0][0]
    vox_idxs.append(max_vox)
  return vox_idxs

In [ ]:
#get coef_m and r2
coef_m = results['COEF_M']
r2 = results['R2']
#get list of n well predicted voxels - n=3 in this case
n = 3
well_predicted_voxs = get_n_well_predicted_voxs(r2, n)
print(n,"best predicted voxels:", well_predicted_voxs)

In [ ]:
coef_m.shape

# Step 3: Inspect the Weights

In [ ]:
#get weights of a single voxel via the voxel number
def get_weights_of_vox(coef_m, vox_num):
  max_vox_coef_m = np.abs(coef_m[vox_num, :])
  return max_vox_coef_m

In [ ]:
###visualizing weights of best predicted voxel###
import matplotlib.pyplot as plt
#max_vox_coef_m = coef_m[well_predicted_voxs[0]] #- no abs value
max_vox_coef_m = get_weights_of_vox(coef_m, well_predicted_voxs[0]) #- w abs value
plt.hist(max_vox_coef_m)
#plt.title('Coef_m of best predicted voxel')
plt.title('Abs Value of Coef_m of best predicted voxel')
plt.show()


In [ ]:
coef_m.shape

In [ ]:
import matplotlib.pyplot as plt
abs_coef_m = np.abs(coef_m)
x = np.mean(abs_coef_m, axis=0)
plt.xticks([])
plt.yticks([])
plt.xlabel('weight values', size=15)
plt.ylabel('frequency of features', size=15)
plt.hist(x)
plt.savefig('ex_weights.png', dpi=250)

In [ ]:
#visualize stats
plt.hist(max_vox_coef_m)
plt.axvspan(.0001,.00044, color='red', alpha=0.5)
plt.show()

In [ ]:
#takes a certain percent (in decimal) of the best weights/features, 
#and returns index of features in list format
def get_percentage_best_features(max_vox_coef_m, decimal_percentage):
  #changes to decimal percentage
  if decimal_percentage > 1:
    decimal_percentage /= 100
  best_features = []
  #sorts coef_m
  max_vox_coef_m_copy = max_vox_coef_m.copy()
  max_vox_coef_m_sorted = np.sort(max_vox_coef_m_copy)
  #takes percent and turns into index at that percent
  percentile = int(max_vox_coef_m.size*decimal_percentage)
  #takes weights from index to highest
  highest_weights = max_vox_coef_m_sorted[-percentile:]
  for weight in highest_weights:
    #gives index of highest weights, appends to list
    loc = np.argwhere(max_vox_coef_m == weight)
    best_feature = loc[0][0]
    best_features.append(best_feature)
  return best_features

In [ ]:
percent_of_highest_weights = .25

In [ ]:
#get max_vox_coef_m of 1st best predicted voxel
print("vox_num1:", well_predicted_voxs[0])
print("percentage_of_highest_weights:", percent_of_highest_weights*100,"%")
max_vox_coef_m1 = get_weights_of_vox(coef_m, well_predicted_voxs[0])
best_features1 = get_percentage_best_features(max_vox_coef_m1, percent_of_highest_weights)
print("total num of features1:", max_vox_coef_m1.size)
print("total num of best_features1:", len(best_features1))

# Step 4: Choose Another Well Predicted Voxel

In [ ]:
#get max_vox_coef_m of 2nd best predicted voxel
print("vox_num2:", well_predicted_voxs[1])
print("percentage_of_highest_weights:", percent_of_highest_weights*100,"%")
max_vox_coef_m2 = get_weights_of_vox(coef_m, well_predicted_voxs[1])
best_features2 = get_percentage_best_features(max_vox_coef_m2, percent_of_highest_weights)
print("total num of features2:", max_vox_coef_m2.size)
print("total num of best_features2:", len(best_features2))

In [ ]:
#get max_vox_coef_m of 3rd best predicted voxel
print("vox_num3:", well_predicted_voxs[2])
print("percentage_of_highest_weights:", percent_of_highest_weights*100,"%")
max_vox_coef_m3 = get_weights_of_vox(coef_m, well_predicted_voxs[2])
best_features3 = get_percentage_best_features(max_vox_coef_m3, percent_of_highest_weights)
print("total num of features3:", max_vox_coef_m3.size)
print("total num of best_features3:", len(best_features3))

Get the shared features

In [ ]:
#counts num of shared features and gives list of those features
#between 2 well-predicted voxels, 1 layer
def get_shared_features(features1, features2):
  shared_features = []
  len_features = len(features1)
  for i in range(len_features):
    if features1[i] in features2:
      shared_feature = features1[i]
      shared_features.append(shared_feature)
  shared_features.sort()
  #return len_features, count, fraction_shared, shared_features
  return shared_features

In [ ]:
#b/t 1 and 2
shared_list1 = get_shared_features(best_features1, best_features2)
print(len(shared_list1))
#b/t 1 and 3
shared_list2 = get_shared_features(best_features1, best_features3)
print(len(shared_list2))
#b/t 2 and 3
shared_list3 = get_shared_features(best_features2, best_features3)
print(len(shared_list3))

In [ ]:
#b/t 1,2, and 3
shared_all = get_shared_features(shared_list1, shared_list3)
len(shared_all)

In [ ]:
#for correlation
def get_weights_of_features(weights, features):
  feature_weights = []
  for i in features:
    feature_weights.append(weights[i])
  return feature_weights

In [ ]:
#v stacks specific vox_nums and their features - can be used for all voxs too
#just set np.arange from 0 to num_voxs
def vstack_vox_and_features(coef_m, vox_nums, features):
  count = 0
  arr = []
  for num in vox_nums:
    weights = get_weights_of_vox(coef_m, num)
    feature_weights = get_weights_of_features(weights, features)
    if count == 0: 
      arr.append(feature_weights)
      count += 1
    else:
      arr = np.vstack((arr, feature_weights))
  return arr

In [ ]:
arr = vstack_vox_and_features(coef_m, well_predicted_voxs, shared_all)

In [ ]:
import seaborn as sns
arr_corrcoef = np.corrcoef(arr)
print("arr.shape:", arr.shape)
print("arr_corrcoef.shape:", arr_corrcoef.shape)
d = sns.heatmap(arr_corrcoef, annot=True)
d.set_title('3 best predicted voxels & their shared features correlation')
plt.show()


In [ ]:
#1) get shared across ALL voxels (use 25% of highest weights) - now most common among all for 25%
#2) get correlation
#3) then set coef_m at those shared features to 0
#4) then run veRSA - see how R2 is affected

In [ ]:
#--- OLD STUFF ---
###1) shared across all 
#num_voxels = results['n_voxels']
#percentage = .50
#print("percentage of highest weighted features:", percentage)
#vox_nums = np.arange(num_voxels)
#all_shared_features = []
#for vox_num in vox_nums:
 # vox_coef_m = get_weights_of_vox(coef_m, vox_num)
  #best_features = get_percentage_best_features(vox_coef_m, percentage)
  #if vox_num == 0:
   # all_shared_features = best_features
  #else:
    #all_shared_features = get_shared_features(all_shared_features, best_features)
  #if vox_num % 10 == 0:
   # print("all_shared_features len at vox num", vox_num, ":", len(all_shared_features))
#print("total num voxels:", len(vox_nums))
###finding: even at 50% highest featuers - no feature is consistent across all vox!#
#-------

In [ ]:
#1) most consistent features at 25% level
#gets counts for most consistent features - each count is at index of feature
def get_feature_counts_for_most_consistent_across_voxels(coef_m, num_voxels, num_features, percentage):
  #0 vector the length of num features
  feature_counts = np.zeros(num_features)
  #list of every vox num
  vox_nums = np.arange(num_voxels)
  #for each voxel
  for vox_num in vox_nums:
    #get weights of vox_num voxel
    vox_coef_m = get_weights_of_vox(coef_m, vox_num)
    #get percentage (25%) highest weighted features - list format
    best_features = get_percentage_best_features(vox_coef_m, percentage)
    for best_feature in best_features:
      #adds 1 to that index every time that the feature is among top 25% for a given voxel
      feature_counts[best_feature] += 1
  #returns array of counts
  return feature_counts

def get_most_common_features_across_voxels(feature_counts, percentage):
  #1D vector of size of features - use for future masking
  feature_1_hot = np.zeros(len(feature_counts))
  #copy and sort
  feature_counts_copy = feature_counts.copy()
  feature_counts_sorted = np.sort(feature_counts_copy)
  #takes percent and turns into index at that percent
  percentile = int(len(feature_counts)*percentage)
  #takes weights from percentile to highest - most common percentage (15%) weighted features
  best_feature_counts = feature_counts_sorted[-percentile:]
  for num_counts in best_feature_counts:
    #locations for which features have that num counts
    locs = np.argwhere(feature_counts == num_counts)
    #for every location with that number of counts - some may have same num counts
    for loc in locs:
      #adds a 1 to features with highest counts
      feature_1_hot[loc] += 1
  #for features w multiple counts - brings them down to 1 for mask
  for i in range(len(feature_1_hot)):
    if feature_1_hot[i] >1:
      feature_1_hot[i] = 1
  feature_idxs = []
  #for features that have a 1 - get the index of those features
  for i in range(len(feature_1_hot)):
    if feature_1_hot[i] == 1:
      feature_idxs.append(i)
  return feature_1_hot, feature_idxs
  

In [ ]:
#getting feature counts
num_voxels = results['n_voxels']
num_features = results['n_features']
percentage = .25
feature_counts = get_feature_counts_for_most_consistent_across_voxels(coef_m, num_voxels,num_features, percentage)

In [ ]:
#visualize feature counts
plt.hist(feature_counts)
plt.axvspan(800, 1300, color='red', alpha=.2)
plt.title('Feature Counts')
plt.show()

In [ ]:
#getting feature_1_hot and getting idxs of those features
percentage = .15
feature_1_hot, feature_idxs = get_most_common_features_across_voxels(feature_counts, percentage)

In [ ]:
#2) get correlation

In [ ]:
import seaborn as sns

#num vox by num features
arr = vstack_vox_and_features(coef_m, np.arange(0, 10), feature_idxs) #from 0 to num_voxels if want all
#num sub by num sub
arr_corr = np.corrcoef(arr)

#visualize RSM
d = sns.heatmap(arr_corr, annot=True)
d.set_title("10 voxels &their shared features correlation")
plt.show()

In [ ]:
#3) get coef_m masking in LOOCV

In [ ]:
def get_coef_m_mask(num_voxels, num_features, feature_1_hot):
  #matrix of ones - size of coef_m
  coef_m_mask = np.ones((num_voxels, num_features))
  for i in range(feature_1_hot.size):
    #if there is a 1 at featuer idx - that feature gets zeroed out in coef_m_mask
    if feature_1_hot[i] == 1:
      coef_m_mask[:,i] = 0
  return coef_m_mask

In [ ]:
coef_m_mask = get_coef_m_mask(num_voxels, num_features, feature_1_hot)
coef_m_mask.shape

In [ ]:
#4) see how veRSA is affected!

In [ ]:
#modified
default_alphas = np.concatenate([np.array([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]), np.logspace(1, 5, 50)])

def modified_leave_one_out_ridge(X, y, coef_m_mask, alphas=default_alphas, fit_intercept=True, normalize=True, mb=None):
    '''
        Construct predicted brain patterns by training on N-1 items, 
        and then predicting the held out item.
        
        X: model responses [numItems x numFeatures]
        y: brain responses [numItems x numVoxels]
    '''
    n_items, n_features = X.shape
    n_voxels = y.shape[1]
    y_pred = np.zeros(y.shape)

    ALPHAS = []
    COEF_M = np.zeros((n_voxels, n_features))
    INTERCEPT = []
    for iter_count, test_idx in enumerate(progress_bar(range(n_items), total=n_items)):
        train_idxs = np.ones(n_items) == True
        train_idxs[test_idx] = False
        test_idxs = ~train_idxs  

        clf = RidgeCV(alphas=alphas, fit_intercept=fit_intercept)

        if normalize:
            scaler = StandardScaler()
            scaler.fit(X[train_idxs])
            X_train = scaler.transform(X[train_idxs])
            X_test = scaler.transform(X[test_idxs])        
        else:
            X_train = X[train_idxs]
            X_test = X[test_idxs]
              
        clf.fit(X_train, y[train_idxs])
        ###change here start
        clf.coef_ *= coef_m_mask
        ###stop
        y_pred[test_idxs] = clf.predict(X_test)
        
        ALPHAS.append(clf.alpha_)
        COEF_M += clf.coef_
        INTERCEPT.append(clf.intercept_)

    ALPHAS = np.stack(ALPHAS)
    COEF_M /= iter_count
    INTERCEPT = np.stack(INTERCEPT)
    R2 = r2_score(y, y_pred, multioutput='raw_values')
    
    return {
        "n_items": n_items,
        "n_features": n_features,
        "n_voxels": y.shape[1],
        "ALPHAS": ALPHAS,
        "COEF_M": COEF_M,
        "INTERCEPT": INTERCEPT,
        "R2": R2,
        "y_pred": y_pred
        #"diff_bt_coef_ms": diff_bt_coef_ms
    }

In [ ]:
#modified
def modified_fit_encoding_model(betas, layer_name, coef_m_mask, model_name='alexnet',
                       dataset='InanimateObjects', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
          
    #print("==> prepare images")
    imgs = prepare_images(dataset=dataset, mean=mean, std=std)
    
    #print("==> load pretrained model")
    model = models.__dict__[model_name](pretrained=True)

    #print("==> extract activation map for the given layer")
    pred_rdms = {}
    feat_rdms = {}
    model.eval()   # <-- very important, freeze normalization stats, no dropout etc.
    with FeatureExtractor(model, [layer_name]) as extractor:
        features = extractor(imgs)
        for layer_name,feat in features.items():
            # retain spatial information, but flatten rows into a 1D feature vector
            X = torch.flatten(feat, 1)
            feat_rdm = 1 - np.corrcoef(X)
            feat_rdms[layer_name] = feat_rdm
            
            #print(f"==> fitting ridge regression model ({layer_name}) (numFeatures={X.shape[1]})")
            #this is the only part changed, modified LOOCV
            results = modified_leave_one_out_ridge(X, betas, coef_m_mask, fit_intercept=True, normalize=False)
            
            # compute the predicted neural RDM
            pred_rdm = 1 - np.corrcoef(results['y_pred'])
            pred_rdms[layer_name] = pred_rdm
              
            # now do something with the rdms, e.g., save them for our split-half analysis
          
    return pred_rdms, feat_rdms, results

Highest

In [ ]:
#getting feature counts for 1 sub
pred_rdms, feat_rdms, results = fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])

num_voxels = results['n_voxels']
num_features = results['n_features']
coef_m = results['COEF_M']
percentage = .25
feature_counts = get_feature_counts_for_most_consistent_across_voxels(coef_m, num_voxels,num_features, percentage)



In [ ]:
percentage = .10
feature_1_hot, feature_idxs = get_most_common_features_across_voxels(feature_counts, percentage)

In [ ]:
class5_feature_idxs = {}
percentages = np.arange(.01 , 1, .01)
for p in percentages:
  feature_1_hot, feature_idxs = get_most_common_features_across_voxels(feature_counts, p)
  class5_feature_idxs[round(p,2)] = feature_idxs

In [ ]:
class5_feature_idxs.keys()

In [ ]:
#getting feature counts for all subs - HERE HERE
subs = [0,1,2,3,4,5,6,7,8,9]
percentage_common = .25

rdms, betas, reliability, image_names = load_brain_data('InanimateObjects')
brain_region = 'EarlyV'
layer_name = 'classifier.5'

for sub in subs:

  sub_betas = betas[brain_region][sub].transpose()
  pred_rdms, feat_rdms, results = fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
  num_voxels = results['n_voxels']
  num_features = results['n_features']
  coef_m = results['COEF_M']
  feature_counts = get_feature_counts_for_most_consistent_across_voxels(coef_m, num_voxels,num_features, percentage_common)
  if sub == 0:
    all_feature_counts = np.zeros(num_features)
    all_feature_counts = all_feature_counts + feature_counts
  else:
    all_feature_counts = all_feature_counts + feature_counts




In [ ]:
class5_feature_idxs2 = {}
percentages = np.arange(.01 , 1, .01)
for percentage in percentages:
  feature_1_hot, feature_idxs = get_most_common_features_across_voxels(all_feature_counts, percentage)
  class5_feature_idxs2[round(percentage,2)] = feature_idxs

In [ ]:
import pickle

def dict_save(dict, file_name):
    with open(file_name + '.pickle', 'wb') as f:
        pickle.dump(dict, f, pickle.HIGHEST_PROTOCOL)

def dict_load(dict_name):
    with open(dict_name , 'rb') as f:
              #+ '.pickle', 'rb') as f:
        return pickle.load(f)

In [ ]:
dict_save(class5_feature_idxs2, 'class5_feature_idxs2')

In [ ]:
feature_counts_x = np.arange(0, len(feature_counts), 1)
plt.hist(feature_counts)

In [ ]:
x = [i for i in feature_counts if i > 1500]
len(x)

In [ ]:
#lesion highest - r2 scores
r2scores = {}
percentages = np.arange(0.05,.3,.05)
for percentage in percentages:
  print('PERCENTAGE:', percentage)
  feature_1_hot, feature_idxs = get_most_common_features_across_voxels(feature_counts, percentage)
  coef_m_mask = get_coef_m_mask(num_voxels, num_features, feature_1_hot)
  pred_rdms, feat_rdms, results = modified_fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   coef_m_mask = coef_m_mask,
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
  r2scores[str(round(percentage,2))] = results['R2']
  print("r2 score mean:", np.mean(results['R2']),"\n\n")



FINAL ANALYSIS

In [ ]:
#lesioning highest - r2 scores - results - FINAL - #EarlyV, InanimateObjects
percentage_common = .25
percentages = np.arange(.01, 1, .01)
subs = [0,1,2,3,4,5,6,7,8,9]
subs = [0,1]
rdms, betas, reliability, image_names = load_brain_data('InanimateObjects')
brain_region = 'EarlyV'
layer_name = 'classifier.5'

no_lesion_r2s = {}
lesioned_r2s = {}
percentage_lesioned = {}

for sub in subs:
  sub_betas = betas[brain_region][sub].transpose()
  pred_rdms, feat_rdms, results = fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name=layer_name,
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
  coef_m = results['COEF_M']
  num_voxels = results['n_voxels']
  num_features = results['n_features']
  original_r2 = results['R2'].mean()
  no_lesion_r2s["sub"+str(sub)+"_"+layer_name] = original_r2
  print("SUB:",str(sub), "original_r2:", original_r2)
  half_original_r2 = original_r2 / 2

  #feature counrs vector
  feature_counts = get_feature_counts_for_most_consistent_across_voxels(coef_m, num_voxels, num_features, percentage_common)

  #1 percent to 100 percent - by increments of 1 percent
  for percentage in percentages:
    feature_1_hot, feature_idxs = get_most_common_features_across_voxels(feature_counts, percentage)
    coef_m_mask = get_coef_m_mask(num_voxels, num_features, feature_1_hot)
    #print("sub"+str(sub)+", percentage"+ str(percentage))
    pred_rdms, feat_rdms, results = modified_fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name=layer_name,
                                                   coef_m_mask = coef_m_mask,
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])

    if results['R2'].mean() <= 0:
      lesioned_r2s["sub"+str(sub)+"_"+layer_name] = results['R2'].mean()
      percentage_lesioned["sub"+str(sub)+"_"+layer_name] = percentage
      print("Sub", str(sub), "Final Percentage:", str(percentage))
      print("Sub", str(sub), "Final R2:", str(results['R2'].mean()))
      break


In [ ]:
no_lesion_r2s.keys()

END FINAL ANALYSIS

In [ ]:
r2scores.keys()

In [ ]:
#visualize r2 and percent lesioned
percentages = [0, 0.05, 0.1 , 0.15, 0.2 , 0.25]
#list with mean of original r2 already there! - append to it!
r2scores_means = [np.mean(r2)]
for p in pcents:
  r2scores_mean = np.mean(r2scores[str(p)])
  r2scores_means.append(r2scores_mean)

plt.scatter(percentages, r2scores_means)
plt.xlabel("percentage of features lesioned")
plt.ylabel("r2 score")
plt.title('highest')
plt.show()

In [ ]:
#lesion highest - group half RSA
percentages = np.arange(0.05,.3,.05)
brain_region = 'EarlyV'
layer_name = "classifier.5"
neural_rdms = rdms[brain_region]
subs = [0,1,2,3,4,5,6,7,8,9]
group_half_rsa_means = np.zeros(percentages.shape)

#i becasuse idx i is ued later!!
for i in range(len(percentages)):
  percentage = percentages[i]
  for sub in subs:
    print("\n\nPercentage:", percentage, ", Sub:", sub)
    sub_betas = betas['EarlyV'][sub].transpose()
    num_voxels = sub_betas.shape[1]
    #num_features = ??? -- have to update this when do for other layers!!!

    #modified LOOCV with lesioning
    feature_1_hot, feature_idxs = get_most_common_features_across_voxels(feature_counts, percentage)
    coef_m_mask = get_coef_m_mask(num_voxels, num_features, feature_1_hot)
    pred_rdms, feat_rdms, results = modified_fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   coef_m_mask = coef_m_mask,
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
    
    #making pred_rdms with 10x72x72
    pred_rdm = pred_rdms[layer_name].reshape((1,72,72))
    feat_rdm = feat_rdms[layer_name].reshape((1,72,72))
    if sub == 0:
      pred_rdms2 = pred_rdm   #pred_rdms1 so that pred_rdms doesnt get relabeled!
      feat_rdms2 = feat_rdm   #feat_rdm1 so that feat_rdms doesnt get relabeled!
    else:
      pred_rdms2 = np.concatenate((pred_rdms2, pred_rdm), axis=0)
      feat_rdms2 = np.concatenate((feat_rdms2, feat_rdm), axis=0)

  print("pred_rdms2.shape", pred_rdms2.shape)

  assert pred_rdms2.shape == neural_rdms.shape, "oops, expected same shapes"

  #getting split halfs
  N = pred_rdms2.shape[0]
  splits = get_split_halves(N)
  print(f"N={N}, num_splits={len(splits)}")

  #getting df
  model_name = "alexnet"
  df = pd.DataFrame(columns=['analysis','model_name','dataset','brain_region',
                           'layer_name', 'split_num', 'split_idxs', 'group', 'pearsonr'])

  for split_num, (group1, group2) in enumerate(progress_bar(splits)):

    idx_group1 = np.array(group1)
    idx_group2 = np.array(group2)                                

    brain_rdm1 = compute_avg_rdm(neural_rdms[idx_group1]).squeeze()
    brain_rdm2 = compute_avg_rdm(neural_rdms[idx_group2]).squeeze()
    
    pred_rdm1 = compute_avg_rdm(pred_rdms2[idx_group1]).squeeze()
    pred_rdm2 = compute_avg_rdm(pred_rdms2[idx_group2]).squeeze()                                

    sim1 = compare_rdms(brain_rdm1, pred_rdm1)
    sim2 = compare_rdms(brain_rdm2, pred_rdm2)

    # record the results
    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group1, 'group1', sim1)

    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group2, 'group2', sim2)
    
  mean, lower, upper = compute_fisherz_ci(df.pearsonr)
  print(f"mean={mean:3.3f}, 95% CI=[{lower:3.3f},{upper:3.3f}]")
  group_half_rsa_means[i] = (mean)



In [ ]:
#visualize group half means and percent lesioned
group_half_means = [0.455, 0.23450298,  0.15100923,  0.06443817,  0.01683275, -0.02467095]
percentages = [0, 0.05, 0.1 , 0.15, 0.2 , 0.25]

plt.scatter(percentages, group_half_means)
plt.title('highest')
plt.yticks(np.arange(0,.6,.1))
plt.xlabel('percetage lesioned')
plt.ylabel('group half rsa mean')
plt.show()

In [ ]:
#maybe most common among all voxels is not a good metric - maybe do highest within voxels for each voxel

Random

In [ ]:
#what happens when lesion a random percentage??

In [ ]:
import random
coef_m_mask = np.ones((num_voxels, num_features))
features = list(np.arange(0,4096))
integer = int(len(features)*.15)
random_features = random.sample(features, integer)
for i in range(len(features)):
  if features[i] in random_features:
    coef_m_mask[:,i] = 0


In [ ]:
import random
def get_random_features_coef_m_mask(num_voxels, num_features, percentage):
  #mask size of coef_m - all ones
  coef_m_mask = np.ones((num_voxels, num_features))
  #0,1,2...,4094,4095
  features = list(np.arange(0,num_features))
  #random sample of percentile many features in features
  percentile = int(len(features)*.15)
  random_features = random.sample(features, percentile)
  #if a feature is in the random features sample, then coef_m mask gets 0 at that col
  for i in range(len(features)):
    if features[i] in random_features:
      coef_m_mask[:,i] = 0
  return coef_m_mask



In [ ]:
#lesion random - r2 scores
randr2scores = {}
features = list(np.arange(0,4096))
percentages = np.arange(0.05,.3,.05)
for percentage in percentages:
  print('PERCENTAGE:', percentage)
  coef_m_mask = get_random_features_coef_m_mask(num_voxels, num_features, percentage)
  #integer = int(len(features)*percentage)
  #random_features = random.sample(features, integer)
  #coef_m_mask = np.ones((num_voxels, num_features))
  #for i in range(len(features)):
    #if features[i] in random_features:
      #coef_m_mask[:,i] = 0
  pred_rdms, feat_rdms, results = modified_fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   coef_m_mask = coef_m_mask,
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
  randr2scores[str(round(percentage,2))] = results['R2']
  print("r2 score mean:", np.mean(results['R2']),"\n\n")

In [ ]:
randr2scores.keys()

In [ ]:
#visualize r2 and percent lesioned
percentages = np.arange(0.05,.3,.05)
randr2scores_means = [np.mean(r2)]
for p in percentages:
  randr2scores_mean = np.mean(randr2scores[str(p)])
  randr2scores_means.append(randr2scores_mean)

In [ ]:
plt.scatter(pcents, randr2scores_means)
plt.xlabel("percentage of features lesioned")
plt.ylabel("r2 score")
plt.title('random')
plt.show()

In [ ]:
#lesion random - group half RSA
brain_region = 'EarlyV'
layer_name = "classifier.5"
neural_rdms = rdms[brain_region]
subs = [0,1,2,3,4,5,6,7,8,9]
percentages = np.arange(0.05,.3,.05)
group_half_rsa_means = np.zeros(percentages.shape)

#i becasuse idx i is ued later!!
for i in range(len(percentages)):
  percentage = percentages[i]
  for sub in subs:
    print("\n\nPercentage:", percentage, ", Sub:", sub)
    sub_betas = betas['EarlyV'][sub].transpose()
    num_voxels = sub_betas.shape[1]
    #num_features = ??? -- have to update this when do for other layers!!!

    #modified LOOCV with lesioning
    coef_m_mask = get_random_features_coef_m_mask(num_voxels, num_features, percentage)
    pred_rdms, feat_rdms, results = modified_fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   coef_m_mask = coef_m_mask,
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
    pred_rdm = pred_rdms[layer_name].reshape((1,72,72))
    feat_rdm = feat_rdms[layer_name].reshape((1,72,72))
    if sub == 0:
      pred_rdms3 = pred_rdm   #pred_rdms1 so that pred_rdms doesnt get relabeled!
      feat_rdms3 = feat_rdm   #feat_rdm1 so that feat_rdms doesnt get relabeled!
    else:
      pred_rdms3 = np.concatenate((pred_rdms3, pred_rdm), axis=0)
      feat_rdms3 = np.concatenate((feat_rdms3, feat_rdm), axis=0)

  print("pred_rdms3.shape", pred_rdms3.shape)

  assert pred_rdms3.shape == neural_rdms.shape, "oops, expected same shapes"

  #getting split halfs
  N = pred_rdms3.shape[0]
  splits = get_split_halves(N)
  print(f"N={N}, num_splits={len(splits)}")

  #getting df
  model_name = "alexnet"
  df = pd.DataFrame(columns=['analysis','model_name','dataset','brain_region',
                           'layer_name', 'split_num', 'split_idxs', 'group', 'pearsonr'])

  for split_num, (group1, group2) in enumerate(progress_bar(splits)):

    idx_group1 = np.array(group1)
    idx_group2 = np.array(group2)                                

    brain_rdm1 = compute_avg_rdm(neural_rdms[idx_group1]).squeeze()
    brain_rdm2 = compute_avg_rdm(neural_rdms[idx_group2]).squeeze()
    
    pred_rdm1 = compute_avg_rdm(pred_rdms3[idx_group1]).squeeze()
    pred_rdm2 = compute_avg_rdm(pred_rdms3[idx_group2]).squeeze()                                

    sim1 = compare_rdms(brain_rdm1, pred_rdm1)
    sim2 = compare_rdms(brain_rdm2, pred_rdm2)

    # record the results
    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group1, 'group1', sim1)

    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group2, 'group2', sim2)
    
  mean, lower, upper = compute_fisherz_ci(df.pearsonr)
  print(f"mean={mean:3.3f}, 95% CI=[{lower:3.3f},{upper:3.3f}]")
  group_half_rsa_means[i] = (mean)





In [ ]:
group_half_rsa_means

In [ ]:
#visualize group_half_means and percentages lesioned 
percentages = [0, 0.05, 0.1 , 0.15, 0.2 , 0.25]
group_half_means = [.455, 0.41972714, 0.43015425, 0.42299897, 0.42655967, 0.43461551]
plt.scatter(percentages, group_half_means)
plt.title('random')
plt.yticks(np.arange(0,.6,.1))
plt.xlabel('percetage lesioned')
plt.ylabel('group half rsa mean')
plt.show()

Lowest

In [ ]:
def get_least_common_features(feature_counts, percentage):
  #1D vector of size of features - use for future masking
  feature_1_hot = np.zeros(len(feature_counts))
  #copy and sort
  feature_counts_copy = feature_counts.copy()
  feature_counts_sorted = np.sort(feature_counts_copy)
  #takes percent and turns into index at that percent
  percentile = int(len(feature_counts)*percentage)
  #takes weights from 0 to percentile - least common percentage(15%) weighted features
  worst_feature_counts = feature_counts_sorted[:percentile]
  #locations for which features have that num counts
  for num_counts in worst_feature_counts:
    locs = np.argwhere(feature_counts == num_counts)
    #for every location with that number of counts - some may have same num counts
    for loc in locs:
      feature_1_hot[loc] += 1
  for i in range(len(feature_1_hot)):
    if feature_1_hot[i] >1:
      feature_1_hot[i] = 1
  feature_idxs = []
  #for features that have a 1 - get the index of those features
  for i in range(len(feature_1_hot)):
    if feature_1_hot[i] == 1:
      feature_idxs.append(i)
  return feature_1_hot, feature_idxs



In [ ]:
#lesion lowest - veRSA r2 scores
lowestr2scores = {}
percentages = np.arange(0.05,.3,.05)
for percentage in percentages:
  least_feature_1_hot, least_feature_idxs = get_least_common_features(feature_counts, percentage)
  coef_m_mask = get_coef_m_mask(num_voxels, num_features, least_feature_1_hot)
  pred_rdms, feat_rdms, results = modified_fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   coef_m_mask = coef_m_mask,
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
  lowestr2scores[str(round(percentage,2))] = results['R2']
  print("r2 score mean:", np.mean(results['R2']),"\n\n")

In [ ]:
lowestr2scores.keys()

In [ ]:
percentages = np.arange(0.05,.3,.05)
#pcents = np.arange(.02,1.02,.02)
lowestr2scores_means = [np.mean(r2)]
for p in pcents:
  lowestr2scores_mean = np.mean(lowestr2scores[str(p)])
  lowestr2scores_means.append(lowestr2scores_mean)

In [ ]:
pcents = ['0.0', '0.01', '0.02', '0.03', '0.04', '0.05', '0.06', '0.07', '0.08', '0.09']
pcents = [float(x) for x in pcents]
pcents

In [ ]:
plt.scatter(pcents, lowestr2scores_means)
plt.xlabel("percentage of features lesioned")
plt.ylabel("r2 score")
plt.title('lowest')
plt.show()

In [ ]:
#lesion lowest - group half RSA
brain_region = 'EarlyV'
layer_name = "classifier.5"
neural_rdms = rdms[brain_region]
subs = [0,1,2,3,4,5,6,7,8,9]
percentages = np.arange(0.05,.3,.05)
group_half_rsa_means = np.zeros(percentages.shape)

#i becasuse idx i is ued later!!
for i in range(len(percentages)):
  percentage = percentages[i]
  for sub in subs:
    print("\n\nPercentage:", percentage, ", Sub:", sub)
    sub_betas = betas['EarlyV'][sub].transpose()
    num_voxels = sub_betas.shape[1]
    #num_features = ??? -- have to update this when do for other layers!!!

    #modified LOOCV with lesioning
    least_feature_1_hot, least_feature_idxs = get_least_common_features(feature_counts, percentage)
    coef_m_mask = get_coef_m_mask(num_voxels, num_features, least_feature_1_hot)
    pred_rdms, feat_rdms, results = modified_fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   coef_m_mask = coef_m_mask,
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
    pred_rdm = pred_rdms[layer_name].reshape((1,72,72))
    feat_rdm = feat_rdms[layer_name].reshape((1,72,72))
    if sub == 0:
      pred_rdms4 = pred_rdm   #pred_rdms1 so that pred_rdms doesnt get relabeled!
      feat_rdms4 = feat_rdm   #feat_rdm1 so that feat_rdms doesnt get relabeled!
    else:
      pred_rdms4 = np.concatenate((pred_rdms4, pred_rdm), axis=0)
      feat_rdms4 = np.concatenate((feat_rdms4, feat_rdm), axis=0)
  print("pred_rdms4.shape", pred_rdms4.shape)

  assert pred_rdms4.shape == neural_rdms.shape, "oops, expected same shapes"

  #getting split halfs
  N = pred_rdms3.shape[0]
  splits = get_split_halves(N)
  print(f"N={N}, num_splits={len(splits)}")

  #getting df
  model_name = "alexnet"
  df = pd.DataFrame(columns=['analysis','model_name','dataset','brain_region',
                           'layer_name', 'split_num', 'split_idxs', 'group', 'pearsonr'])

  for split_num, (group1, group2) in enumerate(progress_bar(splits)):

    idx_group1 = np.array(group1)
    idx_group2 = np.array(group2)                                

    brain_rdm1 = compute_avg_rdm(neural_rdms[idx_group1]).squeeze()
    brain_rdm2 = compute_avg_rdm(neural_rdms[idx_group2]).squeeze()
    
    pred_rdm1 = compute_avg_rdm(pred_rdms4[idx_group1]).squeeze()
    pred_rdm2 = compute_avg_rdm(pred_rdms4[idx_group2]).squeeze()                                

    sim1 = compare_rdms(brain_rdm1, pred_rdm1)
    sim2 = compare_rdms(brain_rdm2, pred_rdm2)

    # record the results
    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group1, 'group1', sim1)

    df = update_df(df, brain_region, model_name, layer_name, split_num, 
                   idx_group2, 'group2', sim2)
    
  mean, lower, upper = compute_fisherz_ci(df.pearsonr)
  print(f"mean={mean:3.3f}, 95% CI=[{lower:3.3f},{upper:3.3f}]")
  group_half_rsa_means[i] = (mean)

In [ ]:
group_half_rsa_means

In [ ]:
#visualize group_half_means and percentages lesioned
percentages = [0, 0.05, 0.1 , 0.15, 0.2 , 0.25]
#original at index 0 - then lesioned ones
group_half_means = [.455, 0.45519225, 0.45519225, 0.45519225, 0.45519225, 0.45519225]
plt.scatter(percentages, group_half_means)
plt.yticks(np.arange(0,.6,.1))
plt.title('lowest')
plt.xlabel('percetage lesioned')
plt.ylabel('group half rsa mean')
plt.show()

In [ ]:
#now: use r2 scores - pseudo code:
#for i in np.arange(.01, 1, .01) (percents used)
#half_r2 = np.r2scores_mean / 2
#if r2 <= half_r2, break
#return percentage...

----------------------------------------------------------------------------------------------------------------------

In [ ]:
#1) features most consistent at 25% level
num_voxels = results['n_voxels']
num_features = results['n_features']
percentage = .25
#feature_counts = np.zeros((num_voxels, num_features))
feature_counts = np.zeros(num_features)

print("percentage of highest weighted features:", percentage)
vox_nums = np.arange(num_voxels)

#goes thru each voxel
for vox_num in vox_nums:
  vox_coef_m = get_weights_of_vox(coef_m, vox_num)
  #gets the percentage (25%) highest weighted features
  best_features = get_percentage_best_features(vox_coef_m, percentage)
  for best_feature in best_features:
    #adds 1 each time that the feature is among top percentage (25%) for a given voxel
    feature_counts[best_feature] += 1

In [ ]:
#15% most common features across all voxels
percentage = .15
#1D vector for future coef_m mask
feature_1_hot = np.zeros(len(feature_counts))
#copy and sort
feature_counts_copy = feature_counts.copy()
feature_counts_sorted = np.sort(feature_counts_copy)
#takes percent and turns into index at that percent
percentile = int(len(feature_counts)*percentage)
#takes weights from percentile to highest - most common 15% weighted features
best_feature_counts = feature_counts_sorted[-percentile:]
#the number of counts at each feature
for num_counts in best_feature_counts:
  #locations for which features have that num counts
  locs = np.argwhere(feature_counts == num_counts)
  for loc in locs:
    #adds a 1 to features with highest counts
    feature_1_hot[loc] += 1



In [ ]:
#for features w multiple counts - brings them down to 1 for mask
for i in range(len(feature_1_hot)):
  if feature_1_hot[i] >1:
    feature_1_hot[i] = 1

In [ ]:
print("feature_1_hot.shape:",  feature_1_hot.shape)
print("feature_1_hot.sum()", feature_1_hot.sum())
print(4096*.15)

In [ ]:
#3) coef__m masking

In [ ]:
# matrix size of coef_m
coef_m_1_hot = np.zeros((coef_m.shape))
coef_m_1_hot.shape

In [ ]:
#if it is a significant feature - the coef_m 0's it out!
for i in range(feature_1_hot.size):
  if feature_1_hot[i] == 1:
    coef_m_1_hot[:, i] = 0

In [ ]:
#use mat mult!!! - 1 hot encoding
#for sig features - have columns in coef_m of that feature = 1
#for non sig features - have cols in coef_m of that feature = 0
#then mat mult - gives what i want!!!!

In [ ]:
#4) see how veRSA is affected!

----------------------------------------------------------------

In [ ]:
pred_rdms, feat_rdms, results1 = fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])

In [ ]:
pred_rdms, feat_rdms, results2 = modified_fit_encoding_model(sub_betas, 
                                                   model_name='alexnet', 
                                                   layer_name='classifier.5',
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])

In [ ]:
results2.keys()

In [ ]:
r2.mean()

In [ ]:
#this is same as original mean
r21 = results1['R2']
r21.mean()

In [ ]:
#this is modified (mask out original)
r22 = results2['R2']
r22.mean()

In [ ]:
#^ comparing R2 scores!!!

Get shared features among all voxels

In [ ]:
x = get_shared_features(best_features0, best_features1)
len(x)

In [ ]:
#gets the weight values of specific features
def get_weights_of_features(weights, features):
  feature_weights = []
  for i in features:
    feature_weights.append(weights[i])
  return feature_weights

Get correlation matrix

In [ ]:
#v stacks specific vox_nums and their features - can be used for all voxs too
#just set np.arange from 0 to num_voxs
def vstack_vox_and_features(vox_nums, features):
  count = 0
  arr = []
  for num in vox_nums:
    weights = get_weights_of_vox(coef_m, num)
    feature_weights = get_weights_of_features(weights, features)
    if count == 0: 
      arr.append(feature_weights)
      count += 1
    else:
      arr = np.vstack((arr, feature_weights))
  return arr

In [ ]:
x = vstack_vox_and_features(well_predicted_voxs, all_shared_features)

In [ ]:
print("shared highest 25% features across all 3 voxels: 484")

In [ ]:
x.shape

In [ ]:
import seaborn as sns
x_corr = np.corrcoef(x)
d = sns.heatmap(x_corr, annot=True)
d.set_title('3 voxels & their shared features correlation')
plt.show()

#--------------------------------------------------------------------------------------------

# Scrap Work for Significance Stats Tests

In [ ]:
#saving a dictionary fct
import pickle

def dict_save(dict, file_name):
    with open(file_name + '.pickle', 'wb') as f:
        pickle.dump(dict, f, pickle.HIGHEST_PROTOCOL)

def dict_load(dict_name):
    with open(dict_name , 'rb') as f:
              #+ '.pickle', 'rb') as f:
        return pickle.load(f)

In [ ]:
###loading file by dragging file from pc to the colab workspace
#EarlyV_coef_m_means = dict_load('EarlyV_coef_m_means.pickle')
#pop_mean = EarlyV_coef_m_means['classifier.5']
#pop_mean

In [ ]:
###loading file w wget
EarlyV_coef_m_means = dict_load('EarlyV_coef_m_means(2).pickle')
EarlyV_coef_m_means.keys()
#pop_mean = EarlyV_coef_m_means['classifier.5']
#pop_mean

In [ ]:
#STATS ANALYSIS
#to get significant features of each 
from scipy import stats

#EarlyV_coef_m_means = dict_load('EarlyV_coef_m_means')
pop_mean = EarlyV_coef_m_means['classifier.5']

#values = np.arange(1000, max_vox_coef_m_sorted.size, 1000)
#print(values)
values = np.arange(0, 4096)
#3 times std - 60,95,99.8 rule??? - think about stats more
max_vox_coef_m_sorted = max_vox_coef_m1.copy()
max_vox_coef_m_sorted = np.sort(max_vox_coef_m_sorted)

print("POP MEAN:", pop_mean)
for idx in range(0, values.size-1):
  subsample = max_vox_coef_m_sorted[values[idx]:values[idx+100]]
  #print("subsample mean:", np.mean(subsample))
  result = stats.ttest_1samp(subsample, pop_mean, alternative='greater')
  if (result.pvalue < .01):
    print("subsample mean:", np.mean(subsample))
    print("subsample (", values[idx],"to", values[idx+100],") p-value:", result.pvalue)
    break

#test = max_vox_coef_m_sorted[]

In [ ]:
#visualization of it
import matplotlib.pyplot as plt
from scipy.stats import norm

x = range(0,50)
y = norm.pdf(x, 0, 1)
plt.plot(x, y)


In [ ]:
#same thing as above - but start from highest to lowest
#to get significant features of each 
from scipy import stats

#EarlyV_coef_m_means = dict_load('EarlyV_coef_m_means')
pop_mean = EarlyV_coef_m_means['classifier.5']


#values = [4095,4094,...,0]
values = list(np.arange(0, 4096))
values.reverse()
#3 times std - 60,95,99.8 rule??? - think about stats more
max_vox_coef_m_sorted = max_vox_coef_m1.copy()
max_vox_coef_m_sorted = np.sort(max_vox_coef_m_sorted)

print("POP MEAN:", pop_mean)
for idx in values:
  subsample = max_vox_coef_m_sorted[idx-10:len(values)]
  print("subsample mean:", np.mean(subsample))
  result = stats.ttest_1samp(subsample, pop_mean, alternative='greater')
  print("results.pvalue:", result.pvalue)
  if (result.pvalue > .07):
    print("subsample mean:", np.mean(subsample))
    #print("subsample (", idx-100, "to", idx,") p-value:", result.pvalue)
    print("subsample (", idx, "to 4096) p-value:", result.pvalue)
    break
percent_features_sig = idx/4096
print("percentage of useful features:", f"{percent_features_sig:.2%}")

#test = max_vox_coef_m_sorted[]

In [ ]:
#3992 to 4092

In [ ]:
sig_features_idxs = []
idxs = np.arange(3992,4096)

for i in idxs:
  coef_m_value = max_vox_coef_m_sorted[i]
  #print("coef_m_value:", coef_m_value)
  idx = np.argwhere(max_vox_coef_m1 == coef_m_value)[0][0]
  sig_features_idxs.append(idx)
  #print(sig_features_idxs)


In [ ]:
best_features_subsample = best_features1[100:]
len(best_features_subsample)
len(sig_features_idxs)

In [ ]:
#mean of weights of each feature
import seaborn as sns
sns.distplot(results['COEF_M'].mean(axis=0))

In [ ]:
np.max(max_vox_coef_m1)

In [ ]:
max_vox_coef_m_copy = max_vox_coef_m1.copy()
max_vox_coef_m_sorted = np.sort(np.abs(max_vox_coef_m_copy))
max_vox_coef_m_sorted_top_20_percent = max_vox_coef_m_sorted[-int(max_vox_coef_m1.size*.20):]
max_vox_coef_m_sorted_top_20_percent.size * 5
#max_vox_coef_m_sorted_top_100
#np.mean(max_vox_coef_m_sorted_top_100)

In [ ]:
EarlyV_coef_m_means = dict_load('EarlyV_coef_m_means(2).pickle')

In [ ]:
pop_mean = EarlyV_coef_m_means['features.3']
pop_mean
# = 2.2886144813910386e-05

In [ ]:
#!pip install --upgrade scipy

#--------------------------------------------------------------------------------------------

# Getting Mean of all Weights from each Layer

In [ ]:
#saving a dictionary fct
import pickle

def dict_save(dict, file_name):
    with open(file_name + '.pickle', 'wb') as f:
        pickle.dump(dict, f, pickle.HIGHEST_PROTOCOL)

def dict_load(dict_name):
    with open(dict_name + '.pickle', 'rb') as f:
        return pickle.load(f)

In [ ]:
EarlyV_coef_m_means = dict_load('EarlyV_coef_m_means(2)')
EarlyV_coef_m_means.keys()

In [ ]:
#get coef_m_mean of a single sub
def get_coef_m_mean(sub, region, layer_name):
  sub_betas = betas[region][sub].transpose()
  pred_rdms, feat_rdms, results = fit_encoding_model(sub_betas,
                                                   #change layer name
                                                   layer_name = layer_name,
                                                   model_name='alexnet',
                                                   dataset='InanimateObjects', 
                                                   mean=[0.485, 0.456, 0.406], 
                                                   std=[0.229, 0.224, 0.225])
  #gets_coef_m from restults - takes abs value - includes neg weights too
  coef_m_mean = np.mean(np.abs(results['COEF_M']))
  return coef_m_mean

#get the average of all coef_ms for a single region
def get_mean_of_all_coef_ms(layer_name, region):
  subs = [0,1,2,3,4,5,6,7,8,9]
  coef_m_means = []
  for sub in subs:
     print("\nSUB:", sub)
     #gets coef_m_mean of a single sub
     coef_m_mean = get_coef_m_mean(sub, region, layer_name)
     print("mean of sub", sub, ":", coef_m_mean)
     #appends to the running list
     coef_m_means.append(coef_m_mean)
     #prints the current mean of the list
     current_mean = np.mean(coef_m_means)
     #current_mean = (np.sum(coef_m_means) / len(coef_m_means))
     print("overall mean at sub", sub, ":", current_mean)
  print("FINAL MEAN:", current_mean)   
  mean_of_all_coef_ms = np.sum(coef_m_means) / len(coef_m_means)
  return coef_m_means, mean_of_all_coef_ms

Taking mean of layers

In [ ]:
def get_layers(model, parent_name='', layer_info=[]):
    for module_name, module in model.named_children():
        layer_name = parent_name + '.' + module_name
        if len(list(module.named_children())):
            layer_info = get_layers(module, layer_name, layer_info=layer_info)
        else:
            layer_info.append(layer_name.strip('.'))
    
    return layer_info
def get_layer_names(model):
    return get_layers(model, parent_name='', layer_info=[])

In [ ]:
model = models.alexnet(pretrained=True)
layer_names = get_layer_names(model)
#layer_names = layer_names[:5] 
layer_names

In [ ]:
#gives layers still needed to run
x = layer_names
y = list(EarlyV_coef_m_means.keys())
layers_needed_to_run = []
for i in x:
  if i not in y:
    layers_needed_to_run.append(i)

layers_needed_to_run

In [ ]:
model = models.alexnet(pretrained=True)
layer_names = get_layer_names(model)
layer_names = layer_names[4] 
layer_names

In [ ]:
model = models.alexnet(pretrained=True)

layer_names = get_layer_names(model)
layer_names = layer_names[4]  
print("layer_names:", layer_names)

#coef_m_means for all layers in EarlyV
#EarlyV_coef_m_means = {}
#EarlyV_coef_m_means = dict_load('EarlyV_coef_m_means')
region = 'EarlyV'
print('\n\n\nBRAIN REGION:', region)
for layer_name in layer_names:
  print('\n\n\nLAYER NAME:', layer_name)
  layer_name = ['features.4']
  coef_m_means, mean_of_all_coef_ms = get_mean_of_all_coef_ms(region = region, layer_name=layer_name)

  EarlyV_coef_m_means[layer_name] = mean_of_all_coef_ms

#save the dict
dict_save(EarlyV_coef_m_means, 'EarlyV_coef_m_means')

In [ ]:
#for a single one:
model = models.alexnet(pretrained=True)

layer_name = 'features.4'
print("layer_name:", layer_name)

region = 'EarlyV'
print('\n\n\nBRAIN REGION:', region)

print('\n\n\nLAYER NAME:', layer_name)

coef_m_means, mean_of_all_coef_ms = get_mean_of_all_coef_ms(region = region, layer_name=layer_name)
EarlyV_coef_m_means[layer_name] = mean_of_all_coef_ms


dict_save(EarlyV_coef_m_means, 'EarlyV_coef_m_means')


In [ ]:
EarlyV_coef_m_means.keys()

In [ ]:
dict_save(EarlyV_coef_m_means, 'EarlyV_coef_m_means')

In [ ]:
from google.colab import files
files.download('EarlyV_coef_m_means.pickle')

In [ ]:
x = dict_load('EarlyV_coef_m_means')
x

In [ ]:
EarlyV_coef_m_means.keys()

In [ ]:
model
#from google.colab import files
#files.download('EarlyV_coef_m_means')

In [ ]:
dict_save(EarlyV_coef_m_means, 'EarlyV_coef_m_means')